In [1]:
### THIS SCRIPT CLEANS THE COLLECTED SEP ###   

In [1]:
import pymongo 
import requests
import json
import pandas as pd
import re

from tqdm import tqdm
from bson.objectid import ObjectId
from pprint import pprint

import lib_sepinpho as sep

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

In [2]:
##### Cleaning Sep_Entries ########
##### Fall 2020 Edition: Completed 10.1.2020 ########
##### Spring 2021 Edition: No Updates ########
##### Summer & Fall 2021 Edition: No Updates ########

#1. Check for entries with InPhO apis.

collection_to_clean = db.sep_entries

sep_no_inpho = pd.DataFrame(collection_to_clean.find( filter={'inpho_api': 'Error: No InPhO entry'},
                                         projection = ['title'],
                                         sort=[('title',1)]
                                        ))
#export to csv to made edits in this file
sep_no_inpho.to_csv('noinpho.csv')


In [5]:
# read the values to update from the CSV and batch update the entries

collection_to_update = db.sep_entries

#import cleaned data
inpho_updates = pd.read_csv('no_inphosjon.csv')

for index,row in tqdm(inpho_updates.iterrows()):
    title = row['title']
    api_endpoint = row['api_endpoint']
    json_type = row['json_type']

    sep.update_sep_json(title,api_endpoint, json_type, collection_to_update)



FileNotFoundError: [Errno 2] File b'no_inphojson.csv' does not exist: b'no_inphojson.csv'

In [3]:
##### Cleaning Sep_Entries ########
##### Summer: Completed 7.16.2020 ########
##### Summer: Completed 10.1.2020 ########

#2. Check for duplicate InPhO JSON data.

collection_to_clean = db.sep_entries_dupe
result = list(collection_to_clean.aggregate([
                                {
                                    '$group' : { '_id' : "$inpho_json.ID",
                                    'count': { '$sum': 1 }
                                    }
                                },
                                {
                                    '$match': { 'count': {'$ne': 1 }}
                                }]))

pprint(result)

[{'_id': None, 'count': 17}]


In [8]:

#Find any article that doesn't have domain info

collection_to_update = db.sep_entries
no_domains = pd.DataFrame(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))
#export to csv to made edits in this file
no_domains.to_csv('no_domains.csv')

In [3]:
#Find any article that doesn't have inpho tag info

collection_to_update = db.sep_entries
no_domains = pd.DataFrame(collection_to_update.find(
    {'inpho_json': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))
#export to csv to made edits in this file
no_domains.to_csv('no_inphosjon.csv')

In [13]:
#update articles in no_domains with new domain data  
#Fall Update: Completed 10.1.2020#
#SPring 2021 Update: Completed 4.3.2021
# Summer and Fall 2021 Updates: Completed 9.27.2021


collection_to_update = db.sep_entries

#import cleaned data
nodmain_updates = pd.read_csv('no_domains.csv')

for index,row in tqdm(nodmain_updates.iterrows()):
    page_url = row['page_url']
    primary_domain = row['primarydomain']
    domain_list = row['domainlist']

    sep.update_domain_info(page_url, domain_list, primary_domain, collection_to_update)

22it [00:00, 215.96it/s]acknowledged: /entries/abu-bakr-al-razi/
 True
[{'_id': ObjectId('615291d29050fcc8b5bd9ed1'), 'title': 'Abu Bakr al-Razi', 'domain_tags': 'Thinker, Philosophy of Religion, Ethics', 'primary_domain': 'Thinker'}]
acknowledged: /entries/cudworth/
 True
[{'_id': ObjectId('615291db9050fcc8b5bd9edc'), 'title': 'Ralph Cudworth', 'domain_tags': 'Thinker, Metaphysics, Epistemology', 'primary_domain': 'Thinker'}]
acknowledged: /entries/fuller-margaret/
 True
[{'_id': ObjectId('615291eb9050fcc8b5bd9eee'), 'title': 'Margaret Fuller', 'domain_tags': 'Thinker, Feminist Philosophy', 'primary_domain': 'Thinker'}]
acknowledged: /entries/hegel-social-political/
 True
[{'_id': ObjectId('615291f09050fcc8b5bd9ef5'), 'title': 'Hegel’s Social and Political Philosophy', 'domain_tags': 'Social and Political Philosophy, Thinker', 'primary_domain': 'Social and Political Philosophy'}]
acknowledged: /entries/hutcheson/
 True
[{'_id': ObjectId('615291f19050fcc8b5bd9ef8'), 'title': 'Francis H

In [15]:
#now test again for no domain info

collection_to_update = db.sep_entries
no_domains = list(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))

pprint(no_domains)

[]


In [17]:
#test for and remove editors notes
#Spring 2021 Update: I manually updated 8 articles in compass to remove the Editors Note from the preamble

collection_to_update = db.sep_entries

editors_notes = list(collection_to_update.find(
    {'preamble_text': {'$regex': '^\['}},
    projection={'page_url':1, 'title':1, 'preamble_text':1, '_id':0}
))
pprint(len(editors_notes))
# pprint(editors_notes)

for article in editors_notes:
    print(article['page_url'])

0
